# K-Means Clustering: Theory and Implementation

## 1. Introduction

K-Means clustering is one of the most fundamental unsupervised machine learning algorithms, used to partition a dataset into $K$ distinct, non-overlapping clusters. The algorithm aims to minimize the within-cluster variance, making it particularly effective for discovering natural groupings in data.

## 2. Mathematical Foundation

### 2.1 Problem Formulation

Given a dataset $\mathbf{X} = \{\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_n\}$ where each $\mathbf{x}_i \in \mathbb{R}^d$, the K-Means algorithm seeks to partition these $n$ observations into $K$ clusters $\mathcal{C} = \{C_1, C_2, \ldots, C_K\}$.

### 2.2 Objective Function

The algorithm minimizes the **Within-Cluster Sum of Squares (WCSS)**, also known as the inertia:

$$J = \sum_{k=1}^{K} \sum_{\mathbf{x}_i \in C_k} \|\mathbf{x}_i - \boldsymbol{\mu}_k\|^2$$

where $\boldsymbol{\mu}_k$ is the centroid (mean) of cluster $C_k$:

$$\boldsymbol{\mu}_k = \frac{1}{|C_k|} \sum_{\mathbf{x}_i \in C_k} \mathbf{x}_i$$

### 2.3 Lloyd's Algorithm

The standard K-Means algorithm follows an iterative two-step procedure:

**Step 1: Assignment** - Assign each point to the nearest centroid:
$$C_k^{(t)} = \{\mathbf{x}_i : \|\mathbf{x}_i - \boldsymbol{\mu}_k^{(t)}\|^2 \leq \|\mathbf{x}_i - \boldsymbol{\mu}_j^{(t)}\|^2 \ \forall j\}$$

**Step 2: Update** - Recalculate centroids:
$$\boldsymbol{\mu}_k^{(t+1)} = \frac{1}{|C_k^{(t)}|} \sum_{\mathbf{x}_i \in C_k^{(t)}} \mathbf{x}_i$$

### 2.4 Convergence Properties

The algorithm is guaranteed to converge because:
1. The objective function $J$ is bounded below by zero
2. Each step (assignment and update) can only decrease or maintain $J$
3. There are finitely many possible partitions

However, convergence is only to a **local minimum**, not necessarily the global optimum.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib for publication-quality figures
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

## 3. Data Generation

We generate synthetic data from a mixture of Gaussian distributions to create well-separated clusters. Each cluster follows:

$$\mathbf{x} \sim \mathcal{N}(\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$$

where $\boldsymbol{\Sigma}_k = \sigma^2 \mathbf{I}$ for isotropic clusters.

In [ ]:
def generate_clustered_data(n_samples=300, n_clusters=4, cluster_std=0.6):
    """
    Generate synthetic clustered data from Gaussian distributions.
    
    Parameters:
    -----------
    n_samples : int
        Total number of data points
    n_clusters : int
        Number of clusters to generate
    cluster_std : float
        Standard deviation of each cluster
        
    Returns:
    --------
    X : ndarray of shape (n_samples, 2)
        Generated data points
    true_labels : ndarray of shape (n_samples,)
        True cluster assignments
    true_centers : ndarray of shape (n_clusters, 2)
        True cluster centers
    """
    samples_per_cluster = n_samples // n_clusters
    
    # Define cluster centers in a grid pattern
    true_centers = np.array([
        [2.0, 2.0],
        [8.0, 2.0],
        [2.0, 8.0],
        [8.0, 8.0]
    ])[:n_clusters]
    
    X = []
    true_labels = []
    
    for k in range(n_clusters):
        # Generate points from Gaussian distribution
        cluster_points = np.random.randn(samples_per_cluster, 2) * cluster_std + true_centers[k]
        X.append(cluster_points)
        true_labels.extend([k] * samples_per_cluster)
    
    X = np.vstack(X)
    true_labels = np.array(true_labels)
    
    # Shuffle data
    shuffle_idx = np.random.permutation(len(X))
    X = X[shuffle_idx]
    true_labels = true_labels[shuffle_idx]
    
    return X, true_labels, true_centers

# Generate dataset
X, true_labels, true_centers = generate_clustered_data(n_samples=300, n_clusters=4, cluster_std=0.7)
print(f"Dataset shape: {X.shape}")
print(f"Number of clusters: {len(true_centers)}")
print(f"True cluster centers:\n{true_centers}")

## 4. K-Means Implementation

We implement the K-Means algorithm from scratch to understand its inner workings.

In [ ]:
class KMeans:
    """
    K-Means clustering algorithm implementation.
    
    Attributes:
    -----------
    n_clusters : int
        Number of clusters
    max_iter : int
        Maximum number of iterations
    tol : float
        Convergence tolerance
    centroids : ndarray
        Cluster centroids after fitting
    labels : ndarray
        Cluster assignments for each point
    inertia_history : list
        WCSS values at each iteration
    """
    
    def __init__(self, n_clusters=3, max_iter=100, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None
        self.labels = None
        self.inertia_history = []
        self.centroid_history = []
        
    def _init_centroids(self, X):
        """Initialize centroids using K-Means++ method."""
        n_samples = X.shape[0]
        centroids = []
        
        # Choose first centroid randomly
        idx = np.random.randint(n_samples)
        centroids.append(X[idx])
        
        # Choose remaining centroids with probability proportional to D(x)^2
        for _ in range(1, self.n_clusters):
            distances = np.array([min(np.sum((x - c)**2) for c in centroids) for x in X])
            probabilities = distances / distances.sum()
            idx = np.random.choice(n_samples, p=probabilities)
            centroids.append(X[idx])
            
        return np.array(centroids)
    
    def _assign_clusters(self, X):
        """Assign each point to the nearest centroid."""
        distances = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            distances[:, k] = np.sum((X - self.centroids[k])**2, axis=1)
        return np.argmin(distances, axis=1)
    
    def _update_centroids(self, X):
        """Update centroids as the mean of assigned points."""
        new_centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            cluster_points = X[self.labels == k]
            if len(cluster_points) > 0:
                new_centroids[k] = cluster_points.mean(axis=0)
            else:
                # Handle empty cluster by reinitializing
                new_centroids[k] = X[np.random.randint(X.shape[0])]
        return new_centroids
    
    def _compute_inertia(self, X):
        """Compute Within-Cluster Sum of Squares."""
        inertia = 0
        for k in range(self.n_clusters):
            cluster_points = X[self.labels == k]
            if len(cluster_points) > 0:
                inertia += np.sum((cluster_points - self.centroids[k])**2)
        return inertia
    
    def fit(self, X):
        """Fit K-Means to the data."""
        # Initialize centroids
        self.centroids = self._init_centroids(X)
        self.centroid_history = [self.centroids.copy()]
        
        for iteration in range(self.max_iter):
            # Assignment step
            self.labels = self._assign_clusters(X)
            
            # Update step
            new_centroids = self._update_centroids(X)
            
            # Compute inertia
            inertia = self._compute_inertia(X)
            self.inertia_history.append(inertia)
            
            # Check convergence
            centroid_shift = np.sum((new_centroids - self.centroids)**2)
            self.centroids = new_centroids
            self.centroid_history.append(self.centroids.copy())
            
            if centroid_shift < self.tol:
                print(f"Converged after {iteration + 1} iterations")
                break
        else:
            print(f"Reached maximum iterations ({self.max_iter})")
            
        return self
    
    def predict(self, X):
        """Predict cluster labels for new data."""
        return self._assign_clusters(X)

## 5. Running K-Means Clustering

In [ ]:
# Fit K-Means to the data
kmeans = KMeans(n_clusters=4, max_iter=100, tol=1e-6)
kmeans.fit(X)

print(f"\nFinal inertia (WCSS): {kmeans.inertia_history[-1]:.2f}")
print(f"\nLearned centroids:\n{kmeans.centroids}")

## 6. Visualization

We create a comprehensive visualization showing:
1. Original data with true labels
2. K-Means clustering results
3. Convergence of inertia
4. Centroid movement during optimization

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Color maps
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
cmap = ListedColormap(colors)

# Plot 1: Original data with true labels
ax1 = axes[0, 0]
scatter1 = ax1.scatter(X[:, 0], X[:, 1], c=true_labels, cmap=cmap, 
                       alpha=0.6, s=50, edgecolors='k', linewidth=0.5)
ax1.scatter(true_centers[:, 0], true_centers[:, 1], c='black', 
            marker='*', s=300, edgecolors='white', linewidth=2, label='True Centers')
ax1.set_xlabel('$x_1$')
ax1.set_ylabel('$x_2$')
ax1.set_title('Original Data with True Labels')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Plot 2: K-Means clustering results
ax2 = axes[0, 1]
scatter2 = ax2.scatter(X[:, 0], X[:, 1], c=kmeans.labels, cmap=cmap, 
                       alpha=0.6, s=50, edgecolors='k', linewidth=0.5)
ax2.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1], c='black', 
            marker='X', s=300, edgecolors='white', linewidth=2, label='Learned Centroids')
ax2.set_xlabel('$x_1$')
ax2.set_ylabel('$x_2$')
ax2.set_title('K-Means Clustering Results')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Plot 3: Inertia convergence
ax3 = axes[1, 0]
iterations = range(1, len(kmeans.inertia_history) + 1)
ax3.plot(iterations, kmeans.inertia_history, 'b-o', linewidth=2, markersize=6)
ax3.set_xlabel('Iteration')
ax3.set_ylabel('Inertia (WCSS)')
ax3.set_title('Convergence of Objective Function')
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, len(kmeans.inertia_history) + 1)

# Add annotation for final inertia
final_inertia = kmeans.inertia_history[-1]
ax3.annotate(f'Final: {final_inertia:.1f}', 
             xy=(len(kmeans.inertia_history), final_inertia),
             xytext=(len(kmeans.inertia_history) - 2, final_inertia + 50),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=10, color='red')

# Plot 4: Centroid movement
ax4 = axes[1, 1]
ax4.scatter(X[:, 0], X[:, 1], c='lightgray', alpha=0.3, s=30)

# Plot centroid trajectories
centroid_history = np.array(kmeans.centroid_history)
for k in range(kmeans.n_clusters):
    trajectory = centroid_history[:, k, :]
    ax4.plot(trajectory[:, 0], trajectory[:, 1], '-', 
             color=colors[k], linewidth=2, alpha=0.7)
    # Plot initial position
    ax4.scatter(trajectory[0, 0], trajectory[0, 1], 
                c=colors[k], marker='o', s=150, edgecolors='black', 
                linewidth=2, zorder=5)
    # Plot final position
    ax4.scatter(trajectory[-1, 0], trajectory[-1, 1], 
                c=colors[k], marker='X', s=200, edgecolors='black', 
                linewidth=2, zorder=5)

ax4.set_xlabel('$x_1$')
ax4.set_ylabel('$x_2$')
ax4.set_title('Centroid Movement During Optimization')
ax4.grid(True, alpha=0.3)

# Add legend for centroid markers
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', 
           markersize=10, label='Initial'),
    Line2D([0], [0], marker='X', color='w', markerfacecolor='gray', 
           markersize=12, label='Final')
]
ax4.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to 'plot.png'")

## 7. Analysis and Evaluation

### 7.1 Clustering Quality Metrics

We can evaluate clustering quality using several metrics:

**Silhouette Score:** Measures how similar a point is to its own cluster compared to other clusters:

$$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$$

where $a(i)$ is the mean intra-cluster distance and $b(i)$ is the mean nearest-cluster distance.

In [ ]:
def silhouette_score(X, labels):
    """
    Compute the mean silhouette score for all samples.
    """
    n_samples = X.shape[0]
    n_clusters = len(np.unique(labels))
    
    silhouette_vals = []
    
    for i in range(n_samples):
        # Compute a(i): mean distance to points in same cluster
        same_cluster = X[labels == labels[i]]
        if len(same_cluster) > 1:
            a_i = np.mean([np.linalg.norm(X[i] - x) for x in same_cluster if not np.array_equal(x, X[i])])
        else:
            a_i = 0
        
        # Compute b(i): min mean distance to points in other clusters
        b_i = np.inf
        for k in range(n_clusters):
            if k != labels[i]:
                other_cluster = X[labels == k]
                if len(other_cluster) > 0:
                    mean_dist = np.mean([np.linalg.norm(X[i] - x) for x in other_cluster])
                    b_i = min(b_i, mean_dist)
        
        # Compute silhouette
        if max(a_i, b_i) > 0:
            s_i = (b_i - a_i) / max(a_i, b_i)
        else:
            s_i = 0
        
        silhouette_vals.append(s_i)
    
    return np.mean(silhouette_vals)

# Calculate metrics
sil_score = silhouette_score(X, kmeans.labels)
print(f"Silhouette Score: {sil_score:.3f}")
print(f"(Range: -1 to 1, higher is better)")

# Compare centroid distances
print(f"\nCentroid Comparison:")
print(f"{'Cluster':<10} {'True Center':<25} {'Learned Centroid':<25} {'Distance':<10}")
print("-" * 70)

# Match learned centroids to true centers (greedy matching)
used = set()
for k in range(len(true_centers)):
    min_dist = np.inf
    best_match = 0
    for j in range(kmeans.n_clusters):
        if j not in used:
            dist = np.linalg.norm(true_centers[k] - kmeans.centroids[j])
            if dist < min_dist:
                min_dist = dist
                best_match = j
    used.add(best_match)
    print(f"{k:<10} {str(true_centers[k]):<25} {str(np.round(kmeans.centroids[best_match], 2)):<25} {min_dist:.3f}")

## 8. The Elbow Method

The elbow method helps determine the optimal number of clusters by plotting inertia against $K$ and finding the "elbow" point where adding more clusters yields diminishing returns.

In [ ]:
# Elbow method analysis
K_range = range(1, 9)
inertias = []

for k in K_range:
    km = KMeans(n_clusters=k, max_iter=100, tol=1e-6)
    km.fit(X)
    inertias.append(km.inertia_history[-1])

plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=4, color='r', linestyle='--', label='Optimal K=4')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia (WCSS)')
plt.title('Elbow Method for Optimal K')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(K_range)
plt.show()

print("\nInertia values for different K:")
for k, inertia in zip(K_range, inertias):
    print(f"K={k}: {inertia:.2f}")

## 9. Conclusions

### Key Findings

1. **Algorithm Performance**: K-Means successfully identified the 4 clusters in our synthetic dataset, with learned centroids closely matching the true cluster centers.

2. **Convergence**: The algorithm converged quickly, with the objective function $J$ (inertia) decreasing monotonically and stabilizing within a few iterations.

3. **Initialization Importance**: The K-Means++ initialization strategy helped achieve good results by spreading initial centroids apart.

### Limitations

- **Assumes spherical clusters**: K-Means works best with convex, isotropic clusters
- **Sensitive to outliers**: Outliers can significantly affect centroid positions
- **Local optima**: Different initializations may yield different results
- **Requires $K$ specification**: The number of clusters must be determined beforehand

### Extensions

- **K-Means++**: Better initialization (implemented here)
- **Mini-batch K-Means**: For large datasets
- **Soft K-Means (GMM)**: Probabilistic cluster assignments
- **Kernel K-Means**: For non-linearly separable data